In [ ]:
# | default_exp _server

In [ ]:
# | export

import asyncio
import multiprocessing
import signal
import threading
from contextlib import contextmanager
from typing import *

import asyncer
import typer

from fastkafka._components.helpers import _import_from_string
from fastkafka._components.logger import get_logger

In [ ]:
import os
from time import sleep

from pydantic import BaseModel

from fastkafka._components.logger import supress_timestamps
from fastkafka._components.test_dependencies import generate_app_in_tmp
from fastkafka.testing import LocalKafkaBroker

In [ ]:
# | notest

# allows async calls in notebooks

import nest_asyncio

In [ ]:
# | notest

nest_asyncio.apply()

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export


class ServerProcess:
    def __init__(self, app: str, kafka_broker: Optional[str] = None):
        self.app = app
        self.should_exit = False
        self.kafka_broker = kafka_broker

    def run(self) -> None:
        return asyncio.run(self._serve())

    async def _serve(self) -> None:
        self._install_signal_handlers()

        self.application = _import_from_string(self.app)
        if self.kafka_broker is not None:
            self.application.set_kafka_broker(self.kafka_broker)

        async with self.application:
            await self._main_loop()

    def _install_signal_handlers(self) -> None:
        if threading.current_thread() is not threading.main_thread():
            raise RuntimeError()

        loop = asyncio.get_event_loop()

        HANDLED_SIGNALS = (
            signal.SIGINT,  # Unix signal 2. Sent by Ctrl+C.
            signal.SIGTERM,  # Unix signal 15. Sent by `kill <pid>`.
        )

        def handle_exit(sig: int) -> None:
            self.should_exit = True

        for sig in HANDLED_SIGNALS:
            loop.add_signal_handler(sig, handle_exit, sig)

    async def _main_loop(self) -> None:
        while not self.should_exit:
            await asyncio.sleep(0.1)

In [ ]:
# | export

_app = typer.Typer()


@_app.command()
def run_fastkafka_server_process(
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    ),
    kafka_broker: Optional[str] = typer.Option(
        None, help="broker to use from kafka_brokers"
    ),
) -> None:
    ServerProcess(app, kafka_broker).run()

In [ ]:
# | notest

print("WARNING: make sure you save the notebook before running this cell\n")

print("Exporting and installing the new version of the CLI command...")
await asyncio.create_subprocess_exec("nbdev_export")
export_process = await asyncio.create_subprocess_exec("nbdev_export")
await export_process.wait()
assert export_process.returncode == 0

install_process = await asyncio.create_subprocess_exec(
    "pip", "install", "-e", "..[all]"
)
await install_process.wait()
assert install_process.returncode == 0

print("ok")


Exporting and installing the new version of the CLI command...
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///work/fastkafka
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Attempting uninstall: fastkafka
    Found existing installation: fastkafka 0.3.0rc0
    Uninstalling fastkafka-0.3.0rc0:
      Successfully uninstalled fastkafka-0.3.0rc0
  Running setup.py develop for fastkafka
ok


In [ ]:
# | export


async def terminate_asyncio_process(p: asyncio.subprocess.Process) -> None:
    logger.info(f"terminate_asyncio_process(): Terminating the process {p.pid}...")
    # Check if SIGINT already propagated to process
    try:
        await asyncio.wait_for(p.wait(), 1)
        logger.info(
            f"terminate_asyncio_process(): Process {p.pid} was already terminated."
        )
        return
    except asyncio.TimeoutError:
        pass

    for i in range(3):
        p.terminate()
        try:
            await asyncio.wait_for(p.wait(), 10)
            logger.info(f"terminate_asyncio_process(): Process {p.pid} terminated.")
            return
        except asyncio.TimeoutError:
            logger.warning(
                f"terminate_asyncio_process(): Process {p.pid} not terminated, retrying..."
            )

    logger.warning(f"Killing the process {p.pid}...")
    p.kill()
    await p.wait()
    logger.warning(f"terminate_asyncio_process(): Process {p.pid} killed!")

In [ ]:
listener_port = 9099
with generate_app_in_tmp() as app:
    async with LocalKafkaBroker(listener_port=listener_port) as bootstrap_server:
        os.environ["KAFKA_HOSTNAME"], os.environ["KAFKA_PORT"] = bootstrap_server.split(":")
        proc = await asyncio.create_subprocess_exec(
            "run_fastkafka_server_process",
            "--kafka-broker",
            "localhost",
            app,
            stdout=asyncio.subprocess.PIPE,
        )
        sleep(5)
        await terminate_asyncio_process(proc)
        outputs, _ = await proc.communicate()

        print(outputs.decode("utf-8"))
        assert proc.returncode == 0

[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._testing.local_broker: Starting zookeeper...
[INFO] fastkafka._testing.local_broker: Starting kafka...
[INFO] fastkafka._testing.local_broker: Local Kafka broker up and running on 127.0.0.1:9099
[INFO] __main__: terminate_asyncio_process(): Terminating the process 596101...
[INFO] __main__: terminate_asyncio_process(): Process 596101 terminated.
[INFO] fastkafka._application.app: set_kafka_broker() : Setting bootstrap_servers value to '127.0.0.1:9099'
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:9099'}'
[INFO] fastkafka._application.app: _create_producer() : created producer usin

[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 595639 terminated.
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 595266...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 595266 terminated.


In [ ]:
listener_port = 9099
with generate_app_in_tmp() as app:
    async with LocalKafkaBroker(listener_port=listener_port) as bootstrap_server:
        os.environ["KAFKA_HOSTNAME"], os.environ["KAFKA_PORT"] = bootstrap_server.split(":")
        proc = await asyncio.create_subprocess_exec(
            "run_fastkafka_server_process", app, stdout=asyncio.subprocess.PIPE
        )
        sleep(5)
        await terminate_asyncio_process(proc)
        outputs, _ = await proc.communicate()

        print(outputs.decode("utf-8"))
        assert proc.returncode == 0

[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.local_broker: Starting zookeeper...
[INFO] fastkafka._testing.local_broker: Starting kafka...
[INFO] fastkafka._testing.local_broker: Local Kafka broker up and running on 127.0.0.1:9099
[INFO] __main__: terminate_asyncio_process(): Terminating the process 596961...
[INFO] __main__: terminate_asyncio_process(): Process 596961 terminated.
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:9099'}'
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:9099'}'
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:9099'}'
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{

[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 596500 terminated.
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 596129...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 596129 terminated.


In [ ]:
# | export


async def run_fastkafka_server(
    num_workers: int, app: str, kafka_broker: Optional[str] = None
) -> None:
    loop = asyncio.get_event_loop()

    HANDLED_SIGNALS = (
        signal.SIGINT,  # Unix signal 2. Sent by Ctrl+C.
        signal.SIGTERM,  # Unix signal 15. Sent by `kill <pid>`.
    )

    d = {"should_exit": False}

    def handle_exit(sig: int, d: Dict[str, bool] = d) -> None:
        d["should_exit"] = True

    for sig in HANDLED_SIGNALS:
        loop.add_signal_handler(sig, handle_exit, sig)

    async with asyncer.create_task_group() as tg:
        args = [
            "run_fastkafka_server_process",
            app,
        ]
        if kafka_broker is not None:
            args[1:1] = ["--kafka-broker", kafka_broker]
        tasks = [
            tg.soonify(asyncio.create_subprocess_exec)(
                *args,
                stdout=asyncio.subprocess.PIPE,
                stdin=asyncio.subprocess.PIPE,
            )
            for i in range(num_workers)
        ]

    procs = [task.value for task in tasks]

    async def log_output(
        output: Optional[asyncio.StreamReader], pid: int, d: Dict[str, bool] = d
    ) -> None:
        if output is None:
            raise RuntimeError("Expected StreamReader, got None. Is stdout piped?")
        while not output.at_eof():
            outs = await output.readline()
            if outs != b"":
                typer.echo(f"[{pid:03d}]: " + outs.decode("utf-8"), nl=False)

    async with asyncer.create_task_group() as tg:
        for proc in procs:
            tg.soonify(log_output)(proc.stdout, proc.pid)

        while not d["should_exit"]:
            await asyncio.sleep(0.2)

        typer.echo("Starting process cleanup, this may take a few seconds...")
        for proc in procs:
            tg.soonify(terminate_asyncio_process)(proc)

    for proc in procs:
        output, _ = await proc.communicate()
        if output:
            typer.echo(f"[{proc.pid:03d}]: " + output.decode("utf-8"), nl=False)

    returncodes = [proc.returncode for proc in procs]
    if not returncodes == [0] * len(procs):
        typer.secho(
            f"Return codes are not all zero: {returncodes}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(1)

In [ ]:
# | export


@contextmanager
def run_in_process(
    target: Callable[..., Any]
) -> Generator[multiprocessing.Process, None, None]:
    p = multiprocessing.Process(target=target)
    try:
        p.start()
        yield p
    except Exception as e:
        print(f"Exception raised {e=}")
    finally:
        p.terminate()
        p.join()

In [ ]:
# | notest

listener_port = 10000
async with LocalKafkaBroker(listener_port=listener_port) as bootstrap_server:
    os.environ["KAFKA_HOSTNAME"], os.environ["KAFKA_PORT"] = bootstrap_server.split(":")

    with generate_app_in_tmp() as app:

        def run_fastkafka_server_test():
            asyncio.run(run_fastkafka_server(4, app, "localhost"))

        with run_in_process(run_fastkafka_server_test) as p:
            sleep(15)

        assert p.exitcode == 0, p.exitcode
        p.close()

print("ok")

[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.local_broker: Starting zookeeper...
[INFO] fastkafka._testing.local_broker: Starting kafka...
[INFO] fastkafka._testing.local_broker: Local Kafka broker up and running on 127.0.0.1:10000
[597823]: [INFO] fastkafka._application.app: set_kafka_broker() : Setting bootstrap_servers value to '127.0.0.1:10000'
[597825]: [INFO] fastkafka._application.app: set_kafka_broker() : Setting bootstrap_servers value to '127.0.0.1:10000'
[597829]: [INFO] fastkafka._application.app: set_kafka_broker() : Setting bootstrap_servers value to '127.0.0.1:10000'
[597829]: [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:10000'}'
[597823]: [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:10000'}'

[597825]: [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[597823]: [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[597825]: [INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'training_data'})
[597825]: [INFO] aiokafka.consumer.consumer: Subscribed to topic(s): {'training_data'}
[597825]: [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer subscribed.
[597823]: [INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'training_data'})
[597823]: [INFO] aiokafka.consumer.consumer: Subscribed to topic(s): {'training_data'}
[597823]: [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer subscribed.
[597827]: [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[597827]: [INFO] aiokafka.consumer.subscription_state: 

[597829]: [WARNING] aiokafka.cluster: Topic training_data is not available during auto-create initialization
[597827]: [WARNING] aiokafka.cluster: Topic realitime_data is not available during auto-create initialization
[597829]: [ERROR] aiokafka.consumer.group_coordinator: Group Coordinator Request failed: [Error 15] CoordinatorNotAvailableError
[597829]: [WARNING] aiokafka.cluster: Topic realitime_data is not available during auto-create initialization
[597825]: [ERROR] aiokafka.consumer.group_coordinator: Group Coordinator Request failed: [Error 15] CoordinatorNotAvailableError
[597823]: [ERROR] aiokafka.consumer.group_coordinator: Group Coordinator Request failed: [Error 15] CoordinatorNotAvailableError
[597823]: [ERROR] aiokafka.cluster: Topic training_data not found in cluster metadata
[597825]: [WARNING] aiokafka.cluster: Topic training_data is not available during auto-create initialization
[597827]: [ERROR] aiokafka.consumer.group_coordinator: Group Coordinator Request failed: 

[597827]: [INFO] aiokafka.consumer.group_coordinator: Joined group '127.0.0.1:10000_group' (generation 2) with member_id aiokafka-0.8.0-3722acd0-16e9-44f0-8eb7-4a3777d0b9a6
[597823]: [INFO] aiokafka.consumer.group_coordinator: Joined group '127.0.0.1:10000_group' (generation 2) with member_id aiokafka-0.8.0-a8469371-9cd8-4c2c-8997-606291d3ebcd
[597825]: [INFO] aiokafka.consumer.group_coordinator: Successfully synced group 127.0.0.1:10000_group with generation 2
[597825]: [INFO] aiokafka.consumer.group_coordinator: Setting newly assigned partitions {TopicPartition(topic='training_data', partition=0)} for group 127.0.0.1:10000_group
[597825]: [INFO] aiokafka.consumer.group_coordinator: Successfully synced group 127.0.0.1:10000_group with generation 2
[597825]: [INFO] aiokafka.consumer.group_coordinator: Setting newly assigned partitions {TopicPartition(topic='realitime_data', partition=0)} for group 127.0.0.1:10000_group
[597829]: [INFO] aiokafka.consumer.group_coordinator: Successfully 

In [ ]:
# | notest

listener_port = 10000
async with LocalKafkaBroker(listener_port=listener_port) as bootstrap_server:
    os.environ["KAFKA_HOSTNAME"], os.environ["KAFKA_PORT"] = bootstrap_server.split(":")

    with generate_app_in_tmp() as app:

        def run_fastkafka_server_test():
            asyncio.run(run_fastkafka_server(4, app, kafka_broker=None))

        with run_in_process(run_fastkafka_server_test) as p:
            sleep(15)

        assert p.exitcode == 0, p.exitcode
        p.close()

print("ok")

[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.local_broker: Starting zookeeper...
[INFO] fastkafka._testing.local_broker: Starting kafka...
[INFO] fastkafka._testing.local_broker: Local Kafka broker up and running on 127.0.0.1:10000
[599189]: [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:10000'}'
[599187]: [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:10000'}'
[599185]: [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:10000'}'
[599191]: [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:10000'}'
[599187]: [INFO] fastkafka._application.app: _create_producer() : cr

[599189]: [INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'realitime_data'})
[599189]: [INFO] aiokafka.consumer.consumer: Subscribed to topic(s): {'realitime_data'}
[599189]: [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer subscribed.
[599191]: [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[599191]: [INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'realitime_data'})
[599191]: [INFO] aiokafka.consumer.consumer: Subscribed to topic(s): {'realitime_data'}
[599191]: [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer subscribed.
[599187]: [ERROR] aiokafka.consumer.group_coordinator: Group Coordinator Request failed: [Error 15] CoordinatorNotAvailableError
[599191]: [ERROR] aiokafka.consumer.group_coordinator: Group Coordinator Request failed: [Error 15] CoordinatorNotAvailableError
[599191

[599191]: [ERROR] aiokafka.consumer.group_coordinator: Group Coordinator Request failed: [Error 15] CoordinatorNotAvailableError
[599191]: [ERROR] aiokafka.consumer.group_coordinator: Group Coordinator Request failed: [Error 15] CoordinatorNotAvailableError
[599191]: [ERROR] aiokafka.cluster: Topic realitime_data not found in cluster metadata
[599191]: [WARNING] aiokafka.cluster: Topic training_data is not available during auto-create initialization
[599185]: [ERROR] aiokafka.consumer.group_coordinator: Group Coordinator Request failed: [Error 15] CoordinatorNotAvailableError
[599187]: [WARNING] aiokafka.cluster: Topic realitime_data is not available during auto-create initialization
[599185]: [WARNING] aiokafka.cluster: Topic realitime_data is not available during auto-create initialization
[599185]: [ERROR] aiokafka.consumer.group_coordinator: Group Coordinator Request failed: [Error 15] CoordinatorNotAvailableError
[599185]: [WARNING] aiokafka.cluster: Topic training_data is not ava

[599187]: [INFO] aiokafka.consumer.group_coordinator: Setting newly assigned partitions set() for group 127.0.0.1:10000_group
[599187]: [INFO] aiokafka.consumer.group_coordinator: Successfully synced group 127.0.0.1:10000_group with generation 2
[599187]: [INFO] aiokafka.consumer.group_coordinator: Setting newly assigned partitions set() for group 127.0.0.1:10000_group
[599185]: [INFO] aiokafka.consumer.group_coordinator: Successfully synced group 127.0.0.1:10000_group with generation 2
[599185]: [INFO] aiokafka.consumer.group_coordinator: Setting newly assigned partitions {TopicPartition(topic='training_data', partition=0)} for group 127.0.0.1:10000_group
[599191]: [INFO] aiokafka.consumer.group_coordinator: Successfully synced group 127.0.0.1:10000_group with generation 2
[599191]: [INFO] aiokafka.consumer.group_coordinator: Setting newly assigned partitions set() for group 127.0.0.1:10000_group
[599189]: [INFO] aiokafka.consumer.group_coordinator: Successfully synced group 127.0.0.1